In [296]:
from substrateinterface import SubstrateInterface

from scripts.utils.chain_model import Chain
from utils.work_with_data import get_data_from_file

chains_file = get_data_from_file("../chains/v20/chains_dev.json")
chains = [Chain(it) for it in chains_file]

rococo_id = "6408de7737c59c238890533af25896a2c20608d8b380bb01029acb392781063e"
rococo = next((chain for chain in chains if chain.chainId == rococo_id))
rococo_parachains = [chain for chain in chains if chain.parentId == rococo_id]

In [406]:
def dry_run_api_ext(runtime_types_prefix: str):
    return {
        "runtime_api": {
            "DryRunApi": {
                "methods": {
                    "dry_run_call": {
                        "description": "Dry run runtime call",
                        "params": [
                            {
                                "name": "origin_caller",
                                "type": f"{runtime_types_prefix}::OriginCaller"
                            },
                            {
                                "name": "call",
                                "type": "GenericCall"
                            }
                        ],
                        "type": "CallDryRunEffectsResult"
                    },
                    "dry_run_xcm": {
                        "description": "Dry run xcm program",
                        "params": [
                            {
                                "name": "origin_location",
                                "type": "xcm::VersionedLocation"
                            },
                            {
                                "name": "xcm",
                                "type": "xcm::VersionedXcm"
                            }
                        ],
                        "type": "XcmDryRunEffectsResult"
                    },
                },
                "types": {
                    "CallDryRunEffectsResult": {
                        "type": "enum",
                        "type_mapping": [
                            [
                                "Ok",
                                "CallDryRunEffects"
                            ],
                            [
                                "Error",
                                "DryRunEffectsResultErr"
                            ]
                        ]
                    },
                    "DryRunEffectsResultErr": {
                        "type": "enum",
                        "value_list": [
                            "Unimplemented",
                            "VersionedConversionFailed"
                        ]
                    },
                    "CallDryRunEffects": {
                        "type": "struct",
                        "type_mapping": [
                            [
                                "execution_result",
                                "CallDryRunExecutionResult" 
                            ],
                            [
                                "emitted_events",
                                f"Vec<{runtime_types_prefix}::RuntimeEvent>"
                            ],
                            [
                                "local_xcm",
                                "Option<xcm::VersionedXcm>"
                            ],
                            [
                                "forwarded_xcms",
                                "Vec<(xcm::VersionedLocation, Vec<xcm::VersionedXcm>)>"
                            ]
                        ]
                    },
                    "CallDryRunExecutionResult": {
                        "type": "enum",
                        "type_mapping": [
                            [
                                "Ok",
                                "DispatchPostInfo"
                            ],
                            [
                                "Error",
                                "DispatchErrorWithPostInfo"
                            ]
                        ]
                    },
                    "XcmDryRunEffects": {
                        "type": "struct",
                        "type_mapping": [
                            [
                                "execution_result",
                                "staging_xcm::v4::traits::Outcome"
                            ],
                            [
                                "emitted_events",
                                f"Vec<{runtime_types_prefix}::RuntimeEvent>"
                            ],
                            [
                                "forwarded_xcms",
                                "Vec<(xcm::VersionedLocation, Vec<xcm::VersionedXcm>)>"
                            ]
                        ]
                    },
                    "XcmDryRunEffectsResult": {
                        "type": "enum",
                        "type_mapping": [
                            [
                                "Ok",
                                "XcmDryRunEffects"
                            ],
                            [
                                "Error",
                                "DryRunEffectsResultErr"
                            ]
                        ]
                    },
                    "DispatchErrorWithPostInfo": {
                       "type": "struct",
                        "type_mapping": [
                            [
                                "post_info",
                                "DispatchPostInfo" 
                            ],
                            [
                                "error",
                                "sp_runtime::DispatchError"
                            ],
                        ] 
                    },
                    "DispatchPostInfo": {
                        "type": "struct",
                        "type_mapping": [
                            [
                                "actual_weight",
                                "Option<Weight>" 
                            ],
                            [
                                "pays_fee",
                                "frame_support::dispatch::Pays"
                            ],
                        ] 
                    }
                }
            },
        }
    }

In [407]:
def planks(amount: int | float, precision: int=12) -> int:
    return amount * 10**precision

In [408]:
from substrateinterface.utils.ss58 import ss58_decode
from typing import Union, List, Self


class VerionsedXcm:
    unversioned: dict
    versioned: dict
    version: int
    
    @staticmethod
    def default_xcm_version() -> int:
        return 4
    
    def __init__(self, universioned: dict | List):
        self.unversioned = universioned
        self.version = VerionsedXcm.default_xcm_version()
        self.versioned = {f"V{self.version}": universioned}
        
    def __str__(self):
        return str(self.versioned)
    
    def is_v4(self) -> bool:
        return self.version == 4


def multi_location(parents: int, junctions: List[dict]) -> VerionsedXcm:
    interior = "Here"

    if len(junctions) > 0:
        interior_variant = f"X{len(junctions)}"
        
        if VerionsedXcm.default_xcm_version() <= 3 and len(junctions) ==1:
            interior = { interior_variant: junctions[0] }
        else:
            interior = { interior_variant: junctions }

    return VerionsedXcm({"parents": parents, "interior": interior})

def asset_id(location: VerionsedXcm) -> VerionsedXcm: 
    if location.is_v4():
        #
        return location
    else:
        return VerionsedXcm({ "Concrete": location.unversioned })

def asset(location: VerionsedXcm, amount: int) -> VerionsedXcm:
    return VerionsedXcm({ "id": asset_id(location).unversioned, "fun": { "Fungible": amount } })

def assets(location: VerionsedXcm, amount: int) -> VerionsedXcm:
    # We need double list since py-substrate-interface considers MultiAssets to be Tuple(Vec(Asset))
    return VerionsedXcm([[asset(location, amount).unversioned]])

def absolute_location(junctions: List[dict]) -> VerionsedXcm:
    return multi_location(parents=0, junctions=junctions)

def xcm_program(instructions: List[dict]) -> VerionsedXcm:
    # We need double list since py-substrate-interface considers Xcm to be Tuple(Vec(Instruction))
    return VerionsedXcm([instructions])


def buy_execution(fees_asset: VerionsedXcm, weight_limit: str | dict = "Unlimited"):
   return {'BuyExecution': {'fees': fees_asset.unversioned, 'weight_limit': weight_limit}}

def deposit_asset(beneficiary: str):
    return { "DepositAsset": { "assets": {'Wild': {'AllCounted': 1}}, "beneficiary": absolute_location(junctions=[account_junction(beneficiary)]).unversioned } }


def account_junction(account: str, decode: bool = True) -> dict:
    if decode:
        account_id =   "0x" + ss58_decode(account)
    else:
        account_id = account
    
    return { "AccountId32": { "network": None, "id": account_id } }

cacheMissing = object()
    
class ReserveLocation:
    symbol: str
    junctions: List[dict]
    _registry: 'XcmRegistry'
    
    _cached_parachain_id: int | None | cacheMissing = cacheMissing
    
    def __init__(self, symbol: str, junctions: List[dict], registry: 'XcmRegistry'):
        self.symbol = symbol
        self.junctions = junctions
        self._registry = registry
        
    def parachain_id(self) -> int | None:
        if self._cached_parachain_id != cacheMissing:
            return self._cached_parachain_id
        
        computed_value = next((junction["Parachain"] for junction in self.junctions if self.__is_parachain_junction(junction)), None)
        self._cached_parachain_id = computed_value
        
        return computed_value
    
    def reserve_chain(self) -> 'XcmChain':
        parachain_id = self.parachain_id()
        
        if parachain_id is not None:
            return self._registry.get_parachain(parachain_id)
        else:
            return self._registry.relay
    
    def local_junctions(self):
        index_of_parachain_junction = next((i for i, junction in enumerate(self.junctions) if self.__is_parachain_junction(junction)), None)
        if index_of_parachain_junction is not None:
            return self.junctions[index_of_parachain_junction + 1:]
        else:
            return self.junctions
        
    def __is_parachain_junction(self, junction: dict) -> bool:
        return junction["Parachain"] is not None

class ReserveLocations:
    _data: dict[str, List[dict]]
    _registry: 'XcmRegistry'

    def __init__(self, registry: 'XcmRegistry'):
        self._data = self.__get_data()
        self._registry = registry

    def get_reserve(self, symbol: str):
        return ReserveLocation(symbol, self._data[symbol], self._registry)

    def __get_data(self):
        return {
            "ROC": [],
        }
    
    
class XcmRegistry:
    reserves: ReserveLocations
    relay: 'XcmChain'
    parachains: List['XcmChain']
    
    def __init__(self, relay: Chain, parachains: List[Chain]):
        self.reserves = ReserveLocations(self)
        self.relay = XcmChain(relay, self)
        self.parachains = [XcmChain(parachain, self) for parachain in parachains]
        
    def get_parachain(self, parachain_id: int) -> Union['XcmChain', None]:
        return next((parachain for parachain in self.parachains if parachain.parachain_id == parachain_id))
        
xcm_pallet_aliases = ["PolkadotXcm", "XcmPallet"]

class XcmChain:
    chain: Chain
    parachain_id: int | None
    _registry: XcmRegistry

    def __init__(self, chain: Chain, registry: XcmRegistry):
        self.chain = chain
        self._registry = registry

        if chain.parentId is not None:
            self.parachain_id = self.__fetch_parachain_id()
        else:
            self.parachain_id = None

    def sibling_location_of(self, destination_chain: Self, account: str | None = None) -> VerionsedXcm:
        parents = 1 if self.parachain_id is not None else 0

        junctions = []

        if destination_chain.parachain_id is not None:
            junctions.append({"Parachain": destination_chain.parachain_id})
            
        if account is not None:
            junctions.append(account_junction(account))

        return multi_location(parents=parents, junctions=junctions)

    def relative_reserve_location_of(self, token: str) -> VerionsedXcm:
        reserve = self._registry.reserves.get_reserve(token)
        reserve_parachain_id = reserve.parachain_id()
        
        same_chain = reserve_parachain_id == self.parachain_id
        
        if same_chain:
            return multi_location(parents=0, junctions=reserve.local_junctions())
        else:
            return multi_location(parents=1, junctions=reserve.junctions)
        
    def xcm_pallet_alias(self) -> str:
        result = next((candidate for candidate in xcm_pallet_aliases if  self.chain.substrate.get_metadata_module(candidate) is not None), None)
        
        if result is None:
            raise Exception(f"No XcmPallet or its aliases has been found. Searched aliases: {xcm_pallet_aliases}")
        
        return result
        
    def is_system_parachain(self) -> bool:
        return self.parachain_id is not None and 1000 <= self.parachain_id < 2000
    
    def is_relay(self) -> bool:
        return self.parachain_id is None
        
    def transfer_type(self, destination_chain: Self, token: str) -> dict | str:
        reserve = self._registry.reserves.get_reserve(token)
        reserve_parachain_id = reserve.parachain_id()
        
        if self._should_use_teleport(destination_chain):
            return "Teleport"
        elif self.parachain_id == reserve_parachain_id:
            return "LocalReserve"
        elif destination_chain.parachain_id == reserve_parachain_id:
            return "DestinationReserve"
        else:
            reserve_chain = reserve.reserve_chain()
            return { "RemoteReserve": self.sibling_location_of(reserve_chain).versioned }
        
    def _should_use_teleport(self, destination_chain: Self) -> bool:
        to_relay_teleport = self.is_system_parachain() and destination_chain.is_relay()
        from_relay_teleport = self.is_relay() and destination_chain.is_system_parachain()
        
        return to_relay_teleport or from_relay_teleport

    def __fetch_parachain_id(self) -> int:
        return self.chain.substrate.query("ParachainInfo", "ParachainId").value

In [409]:
rococo.create_connection(type_registry=dry_run_api_ext("rococo_runtime"))

for p in rococo_parachains:
    p.create_connection(type_registry=dry_run_api_ext("asset_hub_rococo_runtime"))


Connected to  wss://rococo-rpc.polkadot.io/
Connected to  wss://rococo-asset-hub-rpc.polkadot.io


{'actual_weight': (None, {'ref_time': 'u64', 'proof_size': 'u64'}),
 'pays_fee': ('Yes', 'No')}

In [410]:
registry = XcmRegistry(rococo, rococo_parachains)
rococo_xcm = registry.relay
rococo_parachain_xcm = registry.parachains[0]

In [415]:
from scalecodec import GenericCall, GenericEvent


class XcmSentEvent:
    _attributes: dict
    sent_message: VerionsedXcm
    
    def __init__(self, event_data: dict):
        self._attributes = event_data["attributes"]
        self.sent_message = xcm_program(self.__fix_asymmetrical_decoding(self._attributes["message"]))
        
        
    # Tuple of size 1 decodes to just its value but upon encoding requires a Tuple
    # This is causing just decoded Assets to fail when passing to encode function
    @staticmethod
    def __fix_asymmetrical_decoding(instructions: List[dict]) -> List[dict]:        
        for instruction in instructions:
            if "ReceiveTeleportedAsset" in instruction:
                instruction["ReceiveTeleportedAsset"] = [instruction["ReceiveTeleportedAsset"]]
                
        return instructions
            

def is_call_run_error(execution_result: dict):
    return "Error" in execution_result

def handle_call_run_error_execution_result(chain: XcmChain, execution_result: dict):
    error = execution_result["Error"]["error"]
    
    error_description: str

    if "Module" in error:
        module_error =error["Module"]
        error_index = int(module_error["error"][2:4], 16)

        error = chain.chain.substrate.metadata.get_module_error(module_index=module_error["index"], error_index=error_index)
        error_description = str(error)
    else:
        error_description = str(error)
    
    raise Exception(error_description)

def find_event(events: List, event_module: str, event_name: str) -> dict | None:
    return next((event for event in events if event["module_id"] == event_module and event["event_id"] == event_name), None)

def find_xcm_sent(chain: XcmChain, events: List) -> XcmSentEvent | None:
    event_data = find_event(events, event_module=chain.xcm_pallet_alias(), event_name="Sent")
    if event_data is not None:
        return XcmSentEvent(event_data)
    else:
        return None




def extract_destination_asset(xcm_message: List[dict]) -> dict:
    first_instruction = xcm_message[0]
    
    if "ReceiveTeleportedAsset" in first_instruction:
        return first_instruction["ReceiveTeleportedAsset"][0]
    elif "ReserveAssetDeposited" in first_instruction:
        return first_instruction["ReserveAssetDeposited"][0]
    
    raise Exception("Found no destination assets")

def dry_run_xcm_call(chain: XcmChain, call: GenericCall, sender: str) -> XcmSentEvent:
    origin = {"system": {"Signed": sender}}
    dry_run_effects = chain.chain.substrate.runtime_call(api="DryRunApi", method="dry_run_call", params={"origin_caller": origin, "call": call}).value["Ok"]
    execution_result = dry_run_effects["execution_result"]

    if is_call_run_error(execution_result):
        handle_call_run_error_execution_result(chain, execution_result)
    else:
        events = dry_run_effects["emitted_events"]
        xcm_sent = find_xcm_sent(chain, events)
        
        if xcm_sent is None:
            raise Exception(f"No XcmSent were found, got: {events}")
        
        return xcm_sent
    
def is_xcm_run_error(execution_result: dict):
    return "Incomplete" in execution_result or "Error" in execution_result

def handle_xcm_run_error_execution_result(execution_result: dict):
    if "Incomplete" in execution_result:
        error = execution_result["Incomplete"]["error"]
    elif "Error" in execution_result:
        error = execution_result["Error"]["error"]
    else:
        error = execution_result 
    
    raise Exception(str(error))
        
def dry_run_xcm(chain: XcmChain, xcm: VerionsedXcm, origin: VerionsedXcm) -> [GenericEvent]:
    dry_run_effects = chain.chain.substrate.runtime_call(api="DryRunApi", method="dry_run_xcm", params={"origin_location": origin.versioned, "xcm": xcm.versioned}).value["Ok"]
    execution_result = dry_run_effects["execution_result"]
     
    if is_xcm_run_error(execution_result):
        handle_xcm_run_error_execution_result(execution_result)
    else:
        return dry_run_effects["emitted_events"]
    
def dry_run_intermediate_xcm(chain: XcmChain, xcm: VerionsedXcm, origin: VerionsedXcm) -> XcmSentEvent:
    events = dry_run_xcm(chain, xcm, origin)
    
    xcm_sent = find_xcm_sent(chain, events)
        
    if xcm_sent is None:
        raise Exception(f"No XcmSent were found, got: {events}")
    
    return xcm_sent

def dry_run_final_xcm(chain: XcmChain, xcm: VerionsedXcm, origin: VerionsedXcm):
    events = dry_run_xcm(chain, xcm, origin)
    
    print("Destination chain events: ", events)

     

def dry_run_teleport(
        origin_chain: XcmChain,
        destination_chain: XcmChain,
        token: str,
        sender: str,
        recipient: str,
        amount: int | float
):
    dest = origin_chain.sibling_location_of(destination_chain).versioned
    # beneficiary = absolute_location(junctions=[account_junction(recipient)]).versioned

    token_location_origin = origin_chain.relative_reserve_location_of(token)
    token_location_destination = destination_chain.relative_reserve_location_of(token)
    
    assets_param = assets(token_location_origin, amount=planks(amount)).versioned
    print(f"{assets_param=}")

    asset_transfer_type = origin_chain.transfer_type(destination_chain, token)
    print(f"{asset_transfer_type=}")
    
    remote_fees_id = asset_id(token_location_origin).versioned
    print(f"{remote_fees_id=}")
    
    fees_transfer_type = asset_transfer_type
    print(f"{fees_transfer_type=}")
    
    buy_execution_asset = asset(token_location_destination, amount=planks(amount / 2))
    custom_xcm_on_dest = xcm_program([buy_execution(buy_execution_asset), deposit_asset(recipient)]).versioned
    print(f"{custom_xcm_on_dest=}")
    
    # call = rococo.substrate.compose_call(
    #     call_module="XcmPallet",
    #     call_function="teleport_assets",
    #     call_params={
    #         "dest": dest,
    #         "beneficiary": beneficiary,
    #         "assets": assets_param,
    #         "fee_asset_item": 0
    #     }
    # )
    
    
    call = origin_chain.chain.substrate.compose_call(
        call_module=origin_chain.xcm_pallet_alias(),
        call_function="transfer_assets_using_type_and_then",
        call_params={
            "dest": dest,
            "assets": assets_param,
            "assets_transfer_type": asset_transfer_type,
            "remote_fees_id": remote_fees_id,
            "fees_transfer_type": fees_transfer_type,
            "custom_xcm_on_dest": custom_xcm_on_dest,
            "weight_limit": "Unlimited"
        }
    )
    
    print("\n")
    
    xcm_sent_event = dry_run_xcm_call(origin_chain, call, sender)    
    origin_chain_on_destination = destination_chain.sibling_location_of(origin_chain)
    
    print(f"{xcm_sent_event.sent_message.versioned=}")
    print(f"{origin_chain_on_destination.versioned=}")
    print(f"Teleport successfully initiated on {origin_chain.chain.name}")

    
    print("\n")
    
    dry_run_final_xcm(chain=destination_chain, xcm=xcm_sent_event.sent_message, origin=origin_chain_on_destination)
    print(f"Teleport successfully finished on {destination_chain.chain.name}")
  
roco_para_sender = "5FcRVKgmjMpBismm364VCG51NctJTMibxaL1xF9NnUBYfBei"
rococo_sender = "5DqgU9eK7Jdtc4wfozYMdy5pFtyupnfethvLZ15fgiTZDnkM"

    
dry_run_teleport(
    origin_chain=rococo_parachain_xcm,
    destination_chain=rococo_xcm,
    sender=roco_para_sender,
    recipient=rococo_sender,
    token="ROC",
    amount=1
)




assets_param={'V4': [[{'id': {'parents': 1, 'interior': 'Here'}, 'fun': {'Fungible': 1000000000000}}]]}
asset_transfer_type='Teleport'
remote_fees_id={'V4': {'parents': 1, 'interior': 'Here'}}
fees_transfer_type='Teleport'
custom_xcm_on_dest={'V4': [[{'BuyExecution': {'fees': {'id': {'parents': 0, 'interior': 'Here'}, 'fun': {'Fungible': 500000000000.0}}, 'weight_limit': 'Unlimited'}}, {'DepositAsset': {'assets': {'Wild': {'AllCounted': 1}}, 'beneficiary': {'parents': 0, 'interior': {'X1': [{'AccountId32': {'network': None, 'id': '0x4e899a75c0571af8e2721e8d4c7cfe2206c0e63bc1db88b3e7ef69846d159172'}}]}}}}]]}

xcm_sent_event.sent_message.versioned={'V4': [[{'ReceiveTeleportedAsset': [[{'id': {'parents': 0, 'interior': 'Here'}, 'fun': {'Fungible': 1000000000000}}]]}, 'ClearOrigin', {'BuyExecution': {'fees': {'id': {'parents': 0, 'interior': 'Here'}, 'fun': {'Fungible': 1000000000000}}, 'weight_limit': 'Unlimited'}}, {'BuyExecution': {'fees': {'id': {'parents': 0, 'interior': 'Here'}, 'fun

In [340]:
dhrmp_channels_map = rococo.substrate.query_map(module="Hrmp", storage_function="HrmpChannels")
hrmp_channels = [result[0].value for result in hrmp_channels_map]
hrmp_channels

NameError: name 'hrmp_channels_map' is not defined

In [28]:
chain_id_by_parachain_id = {parachain.substrate.query("ParachainInfo", "ParachainId").value:parachain.chainId for parachain in rococo_parachains}
chain_id_by_parachain_id

{1000: '7c34d42fc815d392057c78b49f2755c753440ccd38bcb0405b3bcfb601d08734'}

In [47]:
from typing import List, Dict, Tuple


def construct_chain_graph(
        relay: Chain,
        parachains: List[chains],
        hrmp_channels: List[Tuple[int, int]],
        chain_id_by_parachain_id: Dict[int, str]
)-> Dict[str, List[str]]:
    result: Dict[str, List[str]] = {}
    
    def add_edge_by_id(origin: str, destination: str):
        edges = result.get(origin)
        if edges is None:
            result[origin] = [destination]
        else:
            edges.append(destination)
            
    def add_edge_by_chain(origin: Chain, destination: Chain):
        add_edge_by_id(origin.chainId, destination.chainId)
   
    
    # relay is accessible from each parachain
    for parachain in parachains:
        add_edge_by_chain(relay, parachain)
        add_edge_by_chain(parachain, relay)
    
    # add all supported channels    
    for channel in hrmp_channels:
        origin = chain_id_by_parachain_id.get(channel["recipient"])
        destination = chain_id_by_parachain_id.get(channel["sender"])
        if origin is None or destination is None:
            continue
            
        add_edge_by_id(origin, destination)
        
    return result

construct_chain_graph(rococo, rococo_parachains, hrmp_channels, chain_id_by_parachain_id)

{'6408de7737c59c238890533af25896a2c20608d8b380bb01029acb392781063e': ['7c34d42fc815d392057c78b49f2755c753440ccd38bcb0405b3bcfb601d08734'],
 '7c34d42fc815d392057c78b49f2755c753440ccd38bcb0405b3bcfb601d08734': ['6408de7737c59c238890533af25896a2c20608d8b380bb01029acb392781063e']}

In [52]:
rococo.substrate.runtime_call(api="DryRunApi", method="dry_run_call")

ValueError: Runtime API Call 'DryRunApi.dry_run_call' not found in registry